In [1]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

### Importação

In [2]:
# Carrega a base de dados contendo os registros de importação de fertilizantes
im=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/i.csv').drop(columns='date')

# Elimina ano incompleto
im=im[im.CO_ANO!=2024]

im=im[im.KG_LIQUIDO>0]

# Modifica unidade de peso
im.KG_LIQUIDO=im.KG_LIQUIDO/10**6
im=im.rename(columns={'KG_LIQUIDO':'kton'})
im.head(1)

,CO_ANO,CO_MES,kton
0,1997,3,7.505776


### Chuvas

In [3]:
ch=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv').drop(columns='date')
ch.tail(1)

,CO_ANO,CO_MES,mm
264,2023,12,78.6


### Funde as tabelas de fertilizantes e chuvas

In [4]:
q=pd.DataFrame()
#q=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv')

from sklearn.ensemble import RandomForestRegressor as RFR
# Define hiperparametro
md=1
# Instância o regresssor
RFR=RFR(max_depth=md, random_state=1)

i=im.copy()
c=ch.copy()

# Determina que o adiantamento dos meses em relação as importações seja igual ao adinatamento de máximo efeito, 
# como determinado no capítulo anterior
ad=7

# Adianta os meses
c.CO_MES=c.CO_MES+ad

# Corriges meses fora da faixa
for adi in range(1,ad+1):
    c.loc[c.CO_MES==12+adi,'CO_ANO']=c.loc[c.CO_MES==12+adi]['CO_ANO']+1
    c.loc[c.CO_MES==12+adi,'CO_MES']=adi

# Funde chuvas com importações
i=i.merge(c)
i.head(2)

,CO_ANO,CO_MES,kton,mm
0,2002,7,83.032266,23.2
1,2002,8,78.232067,0.0
2,2002,9,8.362661,0.0
3,2002,10,46.987128,28.0
4,2002,11,92.415166,17.8
...,...,...,...,...
252,2023,8,96.753794,202.1
253,2023,9,172.570477,19.9
254,2023,10,240.914358,83.2
255,2023,11,90.119779,50.5


In [5]:
# Verifica existencia de registros sem quantidade
i[i.kton==0]

,CO_ANO,CO_MES,kton,mm


In [6]:
# Descarta registros sem quantidade
i=i[i.kton!=0]

Executa os testes

In [7]:
# Recria coluna de data
i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1').astype(int)

# Descarta coluna Ano, substituida por data
i=i.drop(columns='CO_ANO')

# Formata mês como informação categórica
i.CO_MES=i.CO_MES.astype('category')

i=pd.get_dummies(i, prefix_sep='~')


for a in range(1,len(i)):

    for train_size_mes in [10*12]:

        train=i.iloc[:a,:].tail(train_size_mes)
        
        if len(train)>=train_size_mes:

            test=i.iloc[a:a+1,:]
                        
            # Train
            X_train_tratamento=train.drop(columns='kton')
            X_train_testemunha=train.drop(columns=['kton','mm'])
            y_train=train.kton
    
            # Test
            X_test_tratamento  =test.drop(columns='kton')
            X_test_testemunha=test.drop(columns=['kton','mm'])
            y_test=test.kton
                
            RFR.fit(X_train_tratamento,y_train)
            
            test_pred=test     .assign(kton_pred_tratamento  =RFR.predict(X_test_tratamento  ).astype(int))
            
            RFR.fit(X_train_testemunha,y_train)
            
            test_pred=test_pred.assign(kton_pred_testemunha=RFR.predict(X_test_testemunha).astype(int))

            test_pred=test_pred.assign(ad=ad,train_size_anos=train_size_mes/12,md=md)
            
            q=pd.concat([q,test_pred])

q['erro_tratamento']=np.abs(q.kton-q.kton_pred_tratamento)
q['erro_testemunha']=np.abs(q.kton-q.kton_pred_testemunha)

print('\nErro Absoluto Médio do Tratamento',round(q.erro_tratamento.mean(),1))
print('\nErro Absoluto Médio da Testemunha',round(q.erro_testemunha.mean(),1))


Erro Absoluto Médio do Tratamento 39.0

Erro Absoluto Médio da Testemunha 43.9


Acima vemos que o tratamento resultou em menor erro absoluto médio.

> Será que essa superioridade do erro da tratamento em realação a testemunha é estatisticamente significante?
>
> No próximo e último capítulo responderemos a essa pergunta.

In [8]:
# retorna para o formato original
q=pd.concat([q[[x for x in q.columns if x not in q.columns[3:15]]],pd.from_dummies(q[q.columns[3:15]],sep='~')],axis=1)

Salva o resultado dos testes

In [9]:
q.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv', index=False)